In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px 
import plotly.graph_objects as go 
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from colorama import Fore, Style
from IPython.display import display, HTML 
print('Setup Complete')

Setup Complete


In [2]:
data = pd.read_csv('Apple_News_Analyzed.csv')

data.head(5)

,ticker,Date,title,content,Open,High,Low,Close,Adj Close,Volume,...,clean_title,clean_content,title_tokenize,content_tokenize,stopless_title,stopless_content,title_stemmed,content_stemmed,sentiment_title,sentiment_content
0,AAPL,7/23/2012,Summer Heat Scorches Europe And U S,Europe flares as summer heat continues Summer...,21.228571,21.639286,20.989643,21.565357,18.257221,487975600.0,...,summer heat scorches europe and u s,europe flares as summer heat continues summer...,['summer heat scorches europe and u s '],['europe flares as summer heat continues summ...,['summer heat scorches europe and u s '],['europe flares as summer heat continues summ...,['summer heat scorches europe and u s '],['europe flares as summer heat continues summ...,0,1
1,AAPL,7/23/2012,Apple Earnings Preview Quarterly Dip On Deck,Last quarter Apple AAPL reported the 2nd bes...,21.692142,21.774286,21.375357,21.461430,18.169237,565132400.0,...,apple earnings preview quarterly dip on deck,last quarter apple aapl reported the 2nd bes...,['apple earnings preview quarterly dip on dec...,['last quarter apple aapl reported the 2nd b...,['apple earnings preview quarterly dip on dec...,['last quarter apple aapl reported the 2nd b...,['apple earnings preview quarterly dip on dec...,['last quarter apple aapl reported the 2nd b...,0,1
2,AAPL,7/23/2012,Trade Apple After Earnings,It may look like a spider web but the mishmas...,20.516430,20.742857,20.357143,20.534643,17.384607,877312800.0,...,trade apple after earnings,it may look like a spider web but the mishmas...,['trade apple after earnings'],['it may look like a spider web but the mishm...,['trade apple after earnings'],['it may look like a spider web but the mishm...,['trade apple after earn'],['it may look like a spider web but the mishm...,0,1
3,AAPL,7/24/2012,Market Bait And Switch,That is the sound we are going to hear soon fr...,20.705713,20.728571,20.370001,20.531429,17.381897,406632800.0,...,market bait and switch,that is the sound we are going to hear soon fr...,['market bait and switch'],['that is the sound we are going to hear soon ...,['market bait and switch'],['that is the sound we are going to hear soon ...,['market bait and switch'],['that is the sound we are going to hear soon ...,0,-1
4,AAPL,7/27/2012,Will AAPL Fall From The Tree,Apple s AAPL sales for the third quarter mis...,20.536072,20.922501,20.413929,20.898571,17.692715,403936400.0,...,will aapl fall from the tree,apple s aapl sales for the third quarter mis...,['will aapl fall from the tree '],['apple s aapl sales for the third quarter m...,['will aapl fall from the tree '],['apple s aapl sales for the third quarter m...,['will aapl fall from the tree '],['apple s aapl sales for the third quarter m...,0,1


In [3]:
data = data.dropna()
m, n = data.shape

train = data[:1400]
test = data[1400:m]

print('Train shape: ', train.shape)
print('Test shape: ', test.shape)

Train shape:  (1400, 21)
Test shape:  (393, 21)


In [4]:
tss = TimeSeriesSplit(n_splits=5)

predictors = ["Open","High","Low","Close","sentiment_title","sentiment_content",]

def model(model, predictors):
    for train_index, test_index in tss.split(data):
        train = data.iloc[train_index]
        test = data.iloc[test_index]

        predictors = [
            "Open",
            "High",
            "Low",
            "Close",
            "sentiment_title",
            "sentiment_content",
        ]

        model.fit(train[predictors], train["Close"])

In [5]:
def pred(model, predictors):
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index)
    
    return preds

In [6]:
rfr = RandomForestRegressor(n_estimators=100, min_samples_split=50, random_state=1)
model(rfr, predictors)
rfr_preds = pred(rfr, predictors)

rfr_mse = mean_squared_error(test["Close"], rfr_preds)
print("Mean Squared Error in RandomForestRegressor predictions: ", rfr_mse)

Mean Squared Error in RandomForestRegressor predictions:  14.82004145073904


In [7]:
cat = CatBoostRegressor()
model(cat, predictors)
cat_preds = pred(cat, predictors)

cat_mse = mean_squared_error(test["Close"], cat_preds)
print("Mean Squared Error in CatBoostRegressor predictions: ", cat_mse)

Learning rate set to 0.033904
0:	learn: 2.9859839	total: 139ms	remaining: 2m 18s
1:	learn: 2.9039959	total: 140ms	remaining: 1m 9s
2:	learn: 2.8189939	total: 141ms	remaining: 46.9s
3:	learn: 2.7373351	total: 142ms	remaining: 35.4s
4:	learn: 2.6687056	total: 143ms	remaining: 28.5s
5:	learn: 2.5908449	total: 144ms	remaining: 23.9s
6:	learn: 2.5184946	total: 145ms	remaining: 20.6s
7:	learn: 2.4451908	total: 146ms	remaining: 18.1s
8:	learn: 2.3819463	total: 147ms	remaining: 16.1s
9:	learn: 2.3166329	total: 147ms	remaining: 14.6s
10:	learn: 2.2507882	total: 148ms	remaining: 13.3s
11:	learn: 2.1861709	total: 149ms	remaining: 12.3s
12:	learn: 2.1282519	total: 152ms	remaining: 11.5s
13:	learn: 2.0662628	total: 155ms	remaining: 10.9s
14:	learn: 2.0099952	total: 157ms	remaining: 10.3s
15:	learn: 1.9524382	total: 158ms	remaining: 9.71s
16:	learn: 1.8973220	total: 159ms	remaining: 9.19s
17:	learn: 1.8408452	total: 160ms	remaining: 8.72s
18:	learn: 1.7907485	total: 161ms	remaining: 8.3s
19:	learn: 

In [8]:
trace1 = go.Scatter(x=train.index, y=train["Close"], mode="lines", name="Train")
trace2 = go.Scatter(x=test.index, y=test["Close"], mode="lines", name="Val")
trace3 = go.Scatter(x=test.index, y=rfr_preds, mode="lines", name="RFR Predictions")
trace4 = go.Scatter(x=test.index, y=cat_preds, mode="lines", name="CatBoost Predictions")

# Combine traces
fig = go.Figure([trace1, trace2, trace3, trace4])

# Update layout
fig.update_layout(
    title="Model Predictions",
    xaxis=dict(title="Date", tickfont=dict(size=14)),
    yaxis=dict(title="Close Price USD ($)", tickfont=dict(size=14)),
    legend=dict(x=0.02, y=0.98),
    plot_bgcolor="black",
    paper_bgcolor="black",
    font=dict(color="white"),
)

# Show plot
fig.show()